In [5]:
import pandas as pd
import random
import torch
from transformers import pipeline
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
from sentence_transformers import InputExample, SentenceTransformer, losses, evaluation, util
import pandas as pd
from torch import nn
import os

def sample_query_context(row):
    q = row['Question']
    c_true = random.choice(row['Correct Answers'].split(';'))
    c_false = random.choice(row['Incorrect Answers'].split(';'))
    return [q, c_true, c_false]

def query1(q, context):
    s = "{}\n\nReference:\n{}"
    ans1 = generate_text(s.format(q, context), return_full_text=False)[0]["generated_text"]
    return ans1

def query2(q, prev_answer, c):
    s = "{}\n\nInput:\nYou said \"{}\". But this is wrong, the right answer is {}"
    q1 = s.format(q, prev_answer, c)
    return generate_text(q1, return_full_text=False)[0]["generated_text"]

csv_file_path = "TruthfulQA.csv"
out_file_path = "experiment_data.csv"
df = pd.read_csv(csv_file_path)
generate_text = pipeline(model="databricks/dolly-v2-12b", torch_dtype=torch.bfloat16, trust_remote_code=True)



RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 52428800 bytes.

In [2]:


train_data = []
for i in range(len(df.index)):
    test = df.loc[i]
    correct_answers = test['Correct Answers'].split(';')
    q,c_true,c_false = sample_query_context(test)
    answer_right = query1(q, test['Correct Answers'])
    answer_wrong = query1(q, test['Incorrect Answers'])
    answer2_right = query2(q,answer_right, c_false)
    answer2_wrong = query2(q,answer_wrong, c_true)

    train_data.append([i, q, answer_right,c_false,answer2_right])
    train_data.append([i, q, answer_wrong,c_true,answer2_wrong])
   


ValueError: The following `model_kwargs` are not used by the model: ['from_tf'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
train_data = pd.DataFrame(train_data)
train_data.to_csv(out_file_path, index=False)